<a href="https://colab.research.google.com/github/rana14ce/Dataset-of-the-Multinational-Pile/blob/v3/GitHub_XGBoost_with_5_fold_cross_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Evaluate the model on the training set
train_r2 = r2_score(y_train, y_train_pred)
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))

# Evaluate the model on the test set
test_r2 = r2_score(y_test, y_test_pred)
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

print("5-Fold CV RMSE: ", cv_rmse)
print("Mean CV RMSE: ", mean_cv_rmse)
print("Training R^2: ", train_r2)
print("Training RMSE: ", train_rmse)
print("Test R^2: ", test_r2)
print("Test RMSE: ", test_rmse)


5-Fold CV RMSE:  [ 895.54552993  524.27614477 1004.03763886  846.89340688 1002.63850449]
Mean CV RMSE:  854.6782449856652
Training R^2:  0.8834880436500692
Training RMSE:  389.8424850260029
Test R^2:  0.18268139092581626
Test RMSE:  1244.8837023683152


In [2]:
pip install pyswarm


  Preparing metadata (setup.py) ... done
  Created wheel for pyswarm: filename=pyswarm-0.6-py3-none-any.whl size=4464 sha256=986843414ef990839f6514f75a1a4df0d86bbfea56192674bbbc5fdc3f180ec1
  Stored in directory: /root/.cache/pip/wheels/71/67/40/62fa158f497f942277cbab8199b05cb61c571ab324e67ad0d6
Successfully built pyswarm


In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from pyswarm import pso

# Load the dataset
url = 'https://raw.githubusercontent.com/rana14ce/Dataset-of-the-Multinational-Pile/main/Dataset%20of%20the%20Multinational%20Pile.csv'
data = pd.read_csv(url)

# Split the data into features and target
X = data.drop('Qm_KN', axis=1)
y = data['Qm_KN']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the objective function for PSO
def objective_function(params):
    max_depth, gamma, learning_rate, n_estimators = params
    model = XGBRegressor(max_depth=int(max_depth), gamma=gamma, learning_rate=learning_rate, n_estimators=int(n_estimators))
    cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='neg_mean_squared_error')
    mean_rmse = np.mean(np.sqrt(-cv_scores))
    return mean_rmse

# Define the bounds for the hyperparameters
lb = [3, 0, 0.01, 100]  # lower bounds: max_depth, gamma, learning_rate, n_estimators
ub = [7, 0.5, 0.2, 300]  # upper bounds: max_depth, gamma, learning_rate, n_estimators

# Perform PSO
optimal_params, _ = pso(objective_function, lb, ub, swarmsize=20, maxiter=100)
print("Optimal Parameters: ", optimal_params)

# Train the best model with the optimal hyperparameters
best_model = XGBRegressor(max_depth=int(optimal_params[0]), gamma=optimal_params[1],
                          learning_rate=optimal_params[2], n_estimators=int(optimal_params[3]))
best_model.fit(X_train_scaled, y_train)

# Perform 5-Fold Cross-Validation with the best model
cv_scores = cross_val_score(best_model, X_train_scaled, y_train, cv=5, scoring='neg_mean_squared_error')
cv_rmse = np.sqrt(-cv_scores)
mean_cv_rmse = np.mean(cv_rmse)

# Predict on the training set
y_train_pred = best_model.predict(X_train_scaled)

# Predict on the test set
y_test_pred = best_model.predict(X_test_scaled)

# Evaluate the model on the training set
train_r2 = r2_score(y_train, y_train_pred)
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))

# Evaluate the model on the test set
test_r2 = r2_score(y_test, y_test_pred)
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

print("5-Fold CV RMSE: ", cv_rmse)
print("Mean CV RMSE: ", mean_cv_rmse)
print("Training R^2: ", train_r2)
print("Training RMSE: ", train_rmse)
print("Test R^2: ", test_r2)
print("Test RMSE: ", test_rmse)


Stopping search: maximum iterations reached --> 100
Optimal Parameters:  [5.68455021e+00 2.92967948e-01 1.00000000e-02 1.84554509e+02]
5-Fold CV RMSE:  [ 901.86115894  528.10482607  988.74345697  843.03715682 1004.33829471]
Mean CV RMSE:  853.2169787023537
Training R^2:  0.8953221227006719
Training RMSE:  369.51440474199933
Test R^2:  0.18765725340565298
Test RMSE:  1241.088470785522
